# LAB 6: Homography
### AI and Machine Learning // Suchkova Natalia М8О - 114М - 22
01.06.23 @ MAI IT-Center

### The Task
1. Изучить и использовать код по гомографиям
2. Сделать панараму больше чем на 2-х снимках
3. Взять код который занимается триангуляцией на основе афинной гомографии, 
4. Придумать и реализовать метод автоматического триангулирования (по ключивым точкам лица с помощью нейросети, взять шар и по окружности триангулировать) и приминить для своей фотографии.